In [87]:
## Importing Necessary Libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [88]:
## Importing the Datasets
data=pd.read_csv('nyc_taxi_trip_duration.csv')
data= data.iloc[0:50000,]

In [89]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [90]:
data.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

In [91]:
data['store_and_fwd_flag']= 1 * (data.store_and_fwd_flag.values=='Y')
data['trip_duration']= data['trip_duration'].apply(lambda x:x/3600)

In [92]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,0,0.111111
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,0,0.305556
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,0,0.454167
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,0,0.316944
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,0,0.235556


In [93]:
data.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag      int32
trip_duration         float64
dtype: object

In [94]:
data.shape

(50000, 11)

# Checking For NULL Values

In [95]:
data.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

# The most suitable Evaluation Metric here is Root Mean Square Error(RMSE) since our target variable is a continous variable and Root Mean Square Error is most widely used Evaluation Metric for Regression Model.



# Starting of Benchmark Model


In [96]:
# Importing Necessary Libraries

from sklearn.utils import shuffle


In [97]:
data = shuffle(data, random_state=42)


div= int(data.shape[0]/4)


train = data.loc[:3*div+1,:]
test= data.loc[3*div+1:]


In [98]:
test['simple_mean']= train['trip_duration'].mean()
test['simple_mean']

37501    0.27969
9332     0.27969
46084    0.27969
28794    0.27969
14364    0.27969
          ...   
11284    0.27969
44732    0.27969
38158    0.27969
860      0.27969
15795    0.27969
Name: simple_mean, Length: 16335, dtype: float64

In [99]:
from sklearn.metrics import mean_squared_error as MSE

In [100]:
simple_mean_error= np.sqrt(MSE(test['trip_duration'],test['simple_mean']))
simple_mean_error

0.8215135411141647

In [101]:
ven_id =pd.pivot_table(train, values= 'trip_duration', index=['vendor_id'], aggfunc=np.mean)
ven_id

,trip_duration
vendor_id,
1,0.265000
2,0.292327


In [102]:
pass_count=pd.pivot_table(train,values='trip_duration', index=['passenger_count'], aggfunc=np.mean)
pass_count

,trip_duration
passenger_count,
0,0.186574
1,0.279315
2,0.269554
3,0.310019
4,0.285447
5,0.292101
6,0.273302


In [103]:
test['vend_id_mean']= 0


for i in train['vendor_id'].unique():
    test['vend_id_mean'][test['vendor_id'] ==str(i)] = train['trip_duration'][train['vendor_id'] == str(i)].mean()

In [104]:
vend_id_mean=np.sqrt(MSE(test['trip_duration'], test['vend_id_mean']))
vend_id_mean

0.8608422627335721

In [105]:
test['pass_count_mean']=0


for i in train['passenger_count'].unique():
    test['pass_count_mean'][test['passenger_count'] == str(i)] = train['trip_duration'][train['passenger_count'] == str(i)].mean()

In [106]:
pass_count_mean=np.sqrt(MSE(test['trip_duration'], test['pass_count_mean']))
pass_count_mean

0.8608422627335721

# Conclusion of Benchmark Model
1. The Error of simple mean of trip duration is 0.8215135411141647
2. The Error of vendor_id and passenger_count are both Equal which values 0.8608422627335721